# 部署 TVM 模块

TVM 提供了两种使用编译库的方法。

- 将库存储为共享库，并将库动态加载到项目中。
- 以系统模块（module）模式将编译后的库捆绑到项目中。

动态加载更加灵活，可以动态加载新模块。系统模块是更 static 的方法。可以在禁止动态库加载的地方使用系统模块。

In [1]:
import set_env # 加载 TVM Python 环境

/media/pc/data/lxw/ai/tvm


加载 Python 模块：

In [1]:
import tvm
import numpy as np
from tvm import te
from tvm import relay
from pathlib import Path

下面分别以 TVM 和 Relay 为例，说明如何构建 TVM 模块。

## TVM 构建

In [2]:
n = te.var("n")
A = te.placeholder((n,), name="A")
B = te.compute(A.shape, lambda *i: A(*i) + 1.0, name="B")
s = te.create_schedule(B.op)

定义输出库的根目录：

In [6]:
base_dir = Path("libs")
base_dir.mkdir()

编译为动态库：

In [7]:
fadd_dylib = tvm.build(s, [A, B], "llvm", name="addone")
dylib_path = str(base_dir/"test_addone_dll.so")
fadd_dylib.export_library(dylib_path)

在系统库模式下编译库：

In [8]:
fadd_syslib = tvm.build(s, [A, B], "llvm", name="addonesys")
syslib_path = str(base_dir/"test_addone_sys.o")
fadd_syslib.save(syslib_path)

## Relay 构建

定义 Relay 模块：

In [9]:
x = relay.var("x", shape=(2, 2), dtype="float32")
y = relay.var("y", shape=(2, 2), dtype="float32")
params = {"y": np.ones((2, 2), dtype="float32")}
mod = tvm.IRModule.from_expr(relay.Function([x, y], x + y))

构建模块：

In [10]:
compiled_lib = relay.build(mod, tvm.target.Target("llvm"), params=params)

将其导出为共享库：

In [11]:
dylib_path = str(base_dir/"test_relay_add.so")
compiled_lib.export_library(dylib_path)

```{note}
如果正在运行交叉编译，还可以考虑导出到 tar 并稍后调用主机编译器。
```

In [12]:
print(compiled_lib.lib.get_source())

; ModuleID = 'TVMMod'
source_filename = "TVMMod"
target datalayout = "e-m:e-p270:32:32-p271:32:32-p272:64:64-i64:64-f80:128-n8:16:32:64-S128"
target triple = "x86_64-unknown-linux-gnu"

%0 = type { i32*, i32 }
%1 = type { i8*, %2, i32, %3, i64*, i64*, i64 }
%2 = type { i32, i32 }
%3 = type { i8, i8, i16 }
%closure_loop_parallel_ax0 = type { float*, float*, float* }

@__TVMAPISetLastError = linkonce dllexport local_unnamed_addr global void (i8*)* null, align 8
@__TVMBackendParallelLaunch = linkonce dllexport local_unnamed_addr global i32 (i32 (i32, %0*, i8*)*, i8*, i32)* null, align 8
@.str = private constant [75 x i8] c"Assert fail: num_args == 3, tvmgen_default_fused_add: num_args should be 3\00", align 1
@.str.1 = private constant [130 x i8] c"Assert fail: p0_code == 3 or p0_code == 13 or p0_code == 7 or p0_code == 4, tvmgen_default_fused_add: Expect arg[0] to be pointer\00", align 1
@.str.2 = private constant [130 x i8] c"Assert fail: p1_code == 3 or p1_code == 13 or p1_code == 7 or